In [2]:
q="produserid:atlas-dpd-production AND jobname:*p2667* AND NOT jobstatus:closed AND maxpss:>10000"

In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from elasticsearch.helpers import scan
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=180)

In [4]:
ind="jobs2016"

ss = {
  "size": 0,
  "aggs": {
    "2": {
      "terms": {
        "field": "jobstatus", "size": 5,
        "order": { "_count": "desc" }
      },
      "aggs": {
        "3": { "avg": { "field": "wall_time"  }   },
        "4": { "avg": { "field": "nevents"  } },
        "5": { "avg": { "field": "cpuconsumptiontime"  }  },
        "6": { "avg": { "field": "corecount" }  },
        "7": { "avg": { "field": "inputfilebytes" }  },
        "8": { "sum": { "field": "inputfilebytes" }  }
      }
    }
  },
  "query": {
    "filtered": {
      "query": {  "query_string": { "analyze_wildcard": True, "lowercase_expanded_terms": False,  "query": q }  }
    }
  }
}

In [5]:
res= es.search( index=ind, body=ss, analyze_wildcard=True)

# print(res)
for r in res['aggregations']['2']['buckets']:
    cores=r['6']['value']
    wall=r['3']['value']
    events=r['4']['value']
    cpu=r['5']['value']
    
    if events<1: continue
    secPerEvent=wall/events*cores
    cpueff=cpu/(wall*cores)
    print (r['key'], "\tjobs:", r['doc_count'],"\tevents:", events, '\t avg.cores:', cores)
    print ("wall time:", wall, "\twalltime per event per core:", secPerEvent, "[s/ev/core]" ) 
    print ("cpu time :", cpu,  "\tcputtime per event:", cpu/events ,"\tcpu eff.:", cpueff ) 
    print ('--------------------------------------------------------')


In [6]:
ss = {
  "fields": ["wall_time","jobstatus"],
  "query": {
    "filtered": {
      "query": {  "query_string": { "analyze_wildcard": True, "lowercase_expanded_terms": False,  "query": q }  }
    }
  }
}

scroll = scan(es, query=ss, index=ind, scroll='10s')
fi=[]
fa=[]
for res in scroll:
    r=res['fields']
    if r['jobstatus'][0]=='finished':
        fi.append(r['wall_time'][0]/3600)
    elif r['jobstatus'][0]=='failed':
        fa.append(r['wall_time'][0]/3600)


In [9]:
plt.hist([fi,fa],bins=30, log=True, alpha=0.5, stacked=False, label=['finished','failed'])
plt.legend(loc='upper right')
plt.show()

ValueError: Data has no positive values, and therefore can not be log-scaled.

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f96ba189e18> (for post_execute):


ValueError: Data has no positive values, and therefore can not be log-scaled.

ValueError: Data has no positive values, and therefore can not be log-scaled.